In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
# データの読み込み
pf = pd.read_csv("/Users/weishen/Documents/自然言語処理実装/実装新5.16/データセット/dataset.csv")

# テキストとラベルのリストを作成
texts = pf["text"].tolist() # text?
labels = pf["is_rumor"].tolist()

# テキストデータとラベルデータをトレーニング用とテスト用に分割
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# テキストを数値化するためのTokenizerを作成
max_words = len(texts)  # 使用する単語の最大数
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)


In [2]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [3]:
vocab_size = 20000  # 使用する単語の最大数を設定 (調整が必要な場合があります)
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_texts)

In [4]:
# テキストデータを数値列に変換
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [5]:
# パディングして固定長のシーケンスにする
maxlen = 100  # 文章の最大長 (必要に応じて調整)
padded_train_data = pad_sequences(train_sequences, maxlen=maxlen)
padded_test_data = pad_sequences(test_sequences, maxlen=maxlen)

In [6]:
# ラベルをnumpy配列に変換
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [7]:
embedding_dim = 32

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(LSTM(64))  # LSTMのユニット数を増やします
model.add(Dropout(0.2))  # 過学習を防ぐためのDropoutレイヤーを追加
model.add(Dense(1, activation='linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           640000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 664897 (2.54 MB)
Trainable params: 664897 (2.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
model.fit(padded_train_data, train_labels, epochs=5, batch_size=32, validation_split=0.2)  # エポック数とバッチサイズを調整


Epoch 1/5
1249/1249 [==============================] - 29s 22ms/step - loss: nan - acc: 0.7811 - val_loss: nan - val_acc: 0.7802
Epoch 2/5
1249/1249 [==============================] - 28s 23ms/step - loss: nan - acc: 0.7788 - val_loss: nan - val_acc: 0.7802
Epoch 3/5
1249/1249 [==============================] - 28s 23ms/step - loss: nan - acc: 0.7788 - val_loss: nan - val_acc: 0.7802
Epoch 4/5
1249/1249 [==============================] - 28s 23ms/step - loss: nan - acc: 0.7788 - val_loss: nan - val_acc: 0.7802
Epoch 5/5
1249/1249 [==============================] - 29s 23ms/step - loss: nan - acc: 0.7788 - val_loss: nan - val_acc: 0.7802


In [10]:
# テストデータの予測
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=maxlen)
predictions = model.predict(padded_test_sequences)
test_data['predicted_score'] = [score[0] for score in predictions]
test_data.to_csv('lstm_predicted_testdata.csv', index=False)

391/391 [==============================] - 2s 5ms/step


NameError: name 'test_data' is not defined